# An Alternative Analysis of the 2020 Election

### How Would the Outcome Change if States Used a Popular Vote System to Allocate Electoral Votes?

The 2020 US Presidential Election has all but wrapped up, with former Vice President Joe Biden defeating incumbent President Donald Trump via the Electoral College by a (current) count of 279 to 214.  At the time I am writing this, some races have yet to be called, including those in North Carolina, Georgia, Arizona and Alaska.  But Biden's 279 electoral votes are enough to give him a majority of the votes no matter the outcome in those states.


For those unfamiliar with the Electoral College, each state is assigned a certain number of electoral votes that is equal to number of senators and representatives that the state has.  The District of Columbia also has three electoral votes.  Each state has two senators and at least one representative in the House, so the minimum number of votes a state can have is three.  California has the most electoral votes with 55.  There are currently 538 total electoral votes that can be won in the presidential election, meaning that a candidate must win at least 270 to win the election.


Every state, with the exception of Nebraska and Maine, use a *winner take all* system to determine the recipient of their electoral votes.  This means that whichever candidate wins the most votes in each state wins all of that state's electoral votes.  Maine and Nebraska have their own system, which is similar, but is not important for the purpose of this analysis so I will not go into it.


![](election_map.PNG)


This is the current 2020 Presidential Election map from ABC News as of November 9th.  States in blue were won by Joe Biden, states in Red were won by Donald Trump, and states in gray have not yet been determined.  I do not feel like this map tells the entire story though.  As long as a candidate wins a state, even by one vote, the entire electorate votes their way.  But is that the best method for determining a winner?  In 2016, Donald Trump received about 4.5 million votes in the state of California, which was his third highest total of any state, but he lost the state, giving Hillary Clinton all 55 electoral votes.  The same happened to Hillary Clinton in Florida.  She had her third highest vote count of any state in Florida, but lost all 29 electoral votes because Donald Trump carried the state.


But what if states split up their electorate vote based on voting percentages in their state?  For instance, say a state has 12 electorate votes.  And also consider a situation where Donald Trump wins 55% of the vote, and Joe Biden wins 45% (assume no votes Independent for simplicity sake).  Under the current political system, Donald Trump would win all 12 electoral votes, but under my alternative system, Donald Trump would win seven (6.6 rounded up) and Joe Biden would win five.  Using our scenario from above, Donald Trump would have carried 17 votes in California, and Hillary Clinton would have received 14 in Florida.  Would this system have been enough to change the outcome of the election? Maybe I'll explore it in a later analysis.


This makes the system a type of hybrid between the current electoral system (i.e. more populous states will still have more votes/influence over the election) and a popular vote system.  *In this analysis*, I want to see how the outcome of the 2020 US Presidential Election might change using the Popular-Electoral system that I described above.

# Analysis


The first thing we need to do is load up a dataset with the election results.  The data I will be using is a csv that I created using vote counts according to the New York Times as of Monday, November 9th.  These numbers are still unofficial, and I will continue to update them as new data becomes available.

In [1]:
import pandas as pd
import numpy as np

results = pd.read_csv('election_results.csv')
results = results.iloc[:,0:6]
results.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51 entries, 0 to 50
Data columns (total 6 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Code             51 non-null     object
 1   state            51 non-null     object
 2   electoral_votes  51 non-null     int64 
 3   votes_d          51 non-null     int64 
 4   votes_r          51 non-null     int64 
 5   votes_i          51 non-null     int64 
dtypes: int64(4), object(2)
memory usage: 2.5+ KB


In [2]:
results.head()

,Code,state,electoral_votes,votes_d,votes_r,votes_i
0,AL,Alabama,9,843473,1434159,32268
1,AK,Alaska,3,64246,118844,7880
2,AZ,Arizona,11,1644074,1626943,52211
3,AR,Arkansas,6,418018,756995,34267
4,CA,California,55,9325844,4814402,286835


As you can see, the data includes state names and abbreviations (also Washington DC), the number of electoral votes that each state has, the number of votes for the Democratic candidate Joe Biden (*votes_d*), the number of votes for President Donald Trump (*votes_r*), all the sum of all votes cast for an Independent candidate.  For the purpose of this analysis, I am allowing electoral votes to be awarded to independents as a whole if they meet the threshold, as a way to kind of play "spoiler" to, or vulture electoral votes away from, other parties.


Below you can verify that the total number of electoral votes is equal to 538.

In [3]:
results['electoral_votes'].sum()

538

Let's next look at the number of votes each party received total.

In [4]:
votes_by_party = results[['votes_d', 'votes_r', 'votes_i']].sum()
votes_by_party.index = ['Joe Biden', 'Donald Trump', 'Independent']
votes_by_party

Joe Biden       75737619
Donald Trump    71318303
Independent      2663862
dtype: int64

Next we need to add up all the votes in each state, to be able to determine percentages which will be used to delegate electoral votes. 

In [5]:
results['total_votes'] = results[['votes_d', 'votes_r', 'votes_i']].sum(axis=1)

Now we delegate the electoral votes by dividing the number of votes for each candidate by the total number of votes in each state, and multiplying this by the number of electoral votes in the state.  Also, for the purpose of this analysis, I am assuming electoral votes as cast as a whole, and thus we need to round each number to the nearest whole number.

In [6]:
results['ev_d'] = (results['electoral_votes'] * (results['votes_d']/results['total_votes'])).round(0)
results['ev_r'] = (results['electoral_votes'] * (results['votes_r']/results['total_votes'])).round(0)
results['ev_i'] = (results['electoral_votes'] * (results['votes_i']/results['total_votes'])).round(0)

In [7]:
prelim_ev = results[['ev_d','ev_r','ev_i']].sum()
prelim_ev.index = ['Joe Biden EV', 'Donald Trump EV', 'Independent EV']
print(prelim_ev)
print("Total Electoral Votes: ",results[['ev_d','ev_r','ev_i']].sum(axis=1).sum(axis=0))

Joe Biden EV       268.0
Donald Trump EV    262.0
Independent EV       2.0
dtype: float64
Total Electoral Votes:  532.0


Using the new Popular-Electoral method, only 532 of 538 electoral votes have been allocated.  This is due to the rounding that we did in the previous step.  Let's take a look at the states that didn't allocate all of their votes (or allocated too many).

In [8]:
#only 532 votes allocated, need to find states that didn't fully allocate (due to rounding)
not_equal = results['electoral_votes'] != results[['ev_d','ev_r','ev_i']].sum(axis=1)
not_equal_df = results.loc[not_equal,['state','electoral_votes','ev_d','ev_r','ev_i']]
not_equal_df['allocated_ev'] = not_equal_df[['ev_d','ev_r','ev_i']].sum(axis=1)
not_equal_df

,state,electoral_votes,ev_d,ev_r,ev_i,allocated_ev
2,Arizona,11,5.0,5.0,0.0,10.0
14,Indiana,11,4.0,6.0,0.0,10.0
16,Kansas,6,2.0,3.0,0.0,5.0
20,Maryland,10,6.0,3.0,0.0,9.0
32,New York,29,16.0,12.0,0.0,28.0
43,Texas,38,18.0,20.0,1.0,39.0
44,Utah,6,2.0,3.0,0.0,5.0
48,West Virginia,5,1.0,3.0,0.0,4.0


As you can see, Arizona, Indiana, Kansas, Maryland, New York, Utah, and West Virginia still have one electoral vote left to allocate, while Texas allocated one too many.  Now the question becomes how to distribute the remaining votes (and how to take one away from Texas). 


As for the states with votes left to cast, I believe that the best method is to award the extra vote to the *winner of the popular vote in the state*.  I believe that this is the fairest option, and it would reward the candidate for winning the state as a whole, much like the electoral college does now.


For simplicity sake, I believe that the best method for removing a vote from Texas is to take away the Independent electoral vote.  As you saw above, the race is very close between the two candidates (268 to 262 with 6 votes left to be counted), while only two total electoral votes went to Independents.  Thus I do not believe it is fair to arbitrarily take away a vote from Biden or Trump. Also since the Independent category includes all candidates and not just the leading vote getter, I believe that taking the vote away from the Independents is the best option.


Below, I will determine the winner of the popular vote in each state, and assign an extra electoral vote for the states that need it.  Keep in mind, these numbers reflect those available as of November 9th, and could change, especially in a close race that has yet to be called (Arizona).  

In [9]:
#Set baseline for category
results['state_winner'] = 'Republican'

#Change to democrat if they won the state
mask = results['votes_d'] > results['votes_r']
results.loc[mask,'state_winner'] = 'Democrat'

Here, I am just performing an audit on five random states to ensure that the state winner was chosen correctly.

In [10]:
audit = np.random.randint(0, 52, 5)
results.iloc[audit,:]

,Code,state,electoral_votes,votes_d,votes_r,votes_i,total_votes,ev_d,ev_r,ev_i,state_winner
33,NC,North Carolina,15,2658899,2734053,71850,5464802,7.0,8.0,0.0,Republican
30,NJ,New Jersey,14,2200289,1537893,55027,3793209,8.0,6.0,0.0,Democrat
41,SD,South Dakota,3,150475,261108,11095,422678,1.0,2.0,0.0,Republican
31,NM,New Mexico,5,498022,400920,20319,919261,3.0,2.0,0.0,Democrat
6,CT,Connecticut,7,1079122,714144,28529,1821795,4.0,3.0,0.0,Democrat


In [11]:
#Award additional electoral vote to state winner where numbers did not add up because of rounding
d_mask = (results['electoral_votes'] > results[['ev_d','ev_r','ev_i']].sum(axis=1)) & (results['state_winner'] == 'Democrat')
results.loc[d_mask,'ev_d'] = results.loc[d_mask,'ev_d']+1
r_mask = (results['electoral_votes'] > results[['ev_d','ev_r','ev_i']].sum(axis=1)) & (results['state_winner'] == 'Republican')
results.loc[r_mask,'ev_r'] = results.loc[r_mask,'ev_r']+1
results.head()

,Code,state,electoral_votes,votes_d,votes_r,votes_i,total_votes,ev_d,ev_r,ev_i,state_winner
0,AL,Alabama,9,843473,1434159,32268,2309900,3.0,6.0,0.0,Republican
1,AK,Alaska,3,64246,118844,7880,190970,1.0,2.0,0.0,Republican
2,AZ,Arizona,11,1644074,1626943,52211,3323228,6.0,5.0,0.0,Democrat
3,AR,Arkansas,6,418018,756995,34267,1209280,2.0,4.0,0.0,Republican
4,CA,California,55,9325844,4814402,286835,14427081,36.0,18.0,1.0,Democrat


In [12]:
results[['ev_d','ev_r','ev_i']].sum(axis=1).sum()

539.0

In [13]:
mask = results['electoral_votes'] != results[['ev_d','ev_r','ev_i']].sum(axis=1)
results.loc[mask,['state','electoral_votes','ev_d','ev_r','ev_i']]

,state,electoral_votes,ev_d,ev_r,ev_i
43,Texas,38,18.0,20.0,1.0


As you can see, additional votes in Arizona, Indiana, etc. were counted, and the only state left with an imbalance is Texas, where we decided to take away the Independent vote.

In [14]:
#Texas gives out an extra vote due to rounding
#Decide to take independent electoral vote away so the effect on the outcome of the election is minimal
results.loc[results['state'] == 'Texas','ev_i']-= 1
results[results['state']=='Texas']

,Code,state,electoral_votes,votes_d,votes_r,votes_i,total_votes,ev_d,ev_r,ev_i,state_winner
43,TX,Texas,38,5216327,5872362,170479,11259168,18.0,20.0,0.0,Republican


In [15]:
results[['ev_d','ev_r','ev_i']].sum(axis=1).sum()

538.0

In [16]:
final_tally = results[['ev_d','ev_r','ev_i']].sum()
final_tally.index = ['Joe Biden', 'Donald Trump', 'Independent']
final_tally

Joe Biden       271.0
Donald Trump    266.0
Independent       1.0
dtype: float64

And there you have it! Joe Biden (unoffically) has won the presidency using our Popular-Electoral system, defeating incumbent Donald Trump by a margin of 271-266.  However, unlike the current electoral system, the Popular-Electoral system is much more susceptible to change as more votes get counted.  A few thousand votes could shift a candidate over the 0.5 threshold, which means the electoral votes would get rounded up instead of down in a state.  This means that the race is not over until every single vote is counted.  


One thing I am curious about is which date designated a vote for the Independent candidates.

In [17]:
results.loc[results['ev_i'] > 0]

,Code,state,electoral_votes,votes_d,votes_r,votes_i,total_votes,ev_d,ev_r,ev_i,state_winner
4,CA,California,55,9325844,4814402,286835,14427081,36.0,18.0,1.0,Democrat


Unsurprisingly, it is California, whose 55 electoral votes make it much easier for the Independents to meet the threshold for a vote.

# A New Map


The last thing that I want to look at in this analysis is a new version of the map from ABC News that I included above.  That map only included three colors, red, blue, and gray.  But like I said before, it does not tell the whole story of a state.  Below, I have created a map that uses a red to blue scale to give a clearer picture of where each state falls on the political scale.  For the purposes of this map, I have not included independent votes, as I wanted to keep 50% the threshold to flip a state from red to blue.  


Each state is labeled with the percent of votes that were in favor of Joe Biden.  Thus, even red states will be labeled with that number.  Subtract that number from 100 to get the percent of votes in favor of Donald Trump

In [18]:
results['% Democrat'] = (results['votes_d']/(results['votes_r']+results['votes_d'])*100).round(2)

In [19]:
import plotly.express as px
fig = px.choropleth(results,  # Input Pandas DataFrame
                    locations='Code',  # DataFrame column with locations
                    color='% Democrat',  # DataFrame column with color values
                    hover_name='state', # DataFrame column hover info
                    locationmode = 'USA-states', # Set to plot as US States
                    color_continuous_scale = 'rdbu',
                    range_color=(0,100))

fig.update_layout(
    title_text = 'States Based on Voting Margins', # Create a Title
    geo_scope='usa',  # Plot only the USA instead of globe
)
fig.show(renderer='notebook_connected')

# Conclusions


There have been questions about the Electoral College's legitimacy and relevancy for years now.  Many people believe that a new system of voting should be implemented.  I created this analysis to see how a unique, alternative to the Electoral College would play out in the 2020 Presidential Election.  Is it the best method?  Probably not.  Would it ever be implemented?  I don't know, politics is outside my area of expertise.


But I do believe that it provides some advantages.  For one, it eliminates the *winner-take-all* element of the Electoral College.  This makes states like California more accessible to Repulicans, and Texas for Democrats.  You would likely see candidates campaigning more in states other than battleground states like Pennsylvania and Michigan.  Their strategies would have to change because there are now votes to be won in states that were previously inaccessible.


I also believe this method provides more value to a single vote.  Many voters believe that their vote doesn't truly matter in the grand scheme of things, unless they live in battleground states.  Under this system, every vote gets a candidate closer to winning another electoral vote, placing much more emphasis on the value of a single vote.  This could also lead to a change in the way people vote for Independent candidates.  Electoral votes would be much more accessible to Independent candidates under this system, which could lead to more voters giving them a chance.  Or it could go in the opposite direction, where a voter who voted Independent because they thought their vote didn't count might switch over to a Democratic or Republican candidate.


At the end of the day, I do not know if the Popular-Electoral system is a viable alternative to the Electoral College.  I am simply a math guy who wanted to analyze a "What-If" scenario.  Thank you for taking the time to read over my analysis. 


If you have any questions/comments/corrections/suggestions or if you want me to bring this type of experience to YOUR company, please reach out at tylerjfi@umich.edu.